In [ ]:
import numpy as np
import os, h5py, json

from utils import plot_data, return_best_chi2dof, plot_ref_data, BuildSample_DY, normalize, candidate_sigma

In [ ]:
def BuildSample_DY_2(INPUT_PATH, seed, features=[], N_Events=-1, nfiles=20, shuffle=True):
    np.random.seed(seed)
    #random integer to select Zprime file between n files                                                                                                            
    u = np.arange(nfiles)#np.random.randint(100, size=100)                                                                                                           
    if shuffle: np.random.shuffle(u)
    toy_label = INPUT_PATH.split("/")[-2]
    print(toy_label)
    HLF = np.array([])
    for u_i in u:
        if not os.path.exists(INPUT_PATH+toy_label+str(u_i+1)+".h5"): continue
        f    = h5py.File(INPUT_PATH+toy_label+str(u_i+1)+".h5", 'r')
        keys = list(f.keys())
        if u_i==u[0]:
            print('available features: ', keys)
        if len(keys)==0: continue #check whether the file is empty                                                                                                   
        cols = np.array([])
        if len(features): keys = features
        for i in range(len(keys)):
            feature = np.array(f.get(keys[i]))
            feature = np.expand_dims(feature, axis=1)
            if i==0: cols = feature
            else: cols = np.concatenate((cols, feature), axis=1)
        if shuffle: np.random.shuffle(cols) #don't want to select always the same event first                                                                       \
                                                                                                                                                                     
        if HLF.shape[0]==0:
            HLF=cols
            i+=1
        else: HLF=np.concatenate((HLF, cols), axis=0)
        f.close()
        if N_Events>0 and HLF.shape[0]>=N_Events:
            HLF=HLF[:N_Events, : ]
            break
    print(HLF.shape)
    #return HLF[:, [4, 5, 1, 2, 0, 3]]                                                                                                                               
    return HLF

In [ ]:
cut_mll = 60
cut_pt = 20
cut_eta = 2.4

In [ ]:
#['delta_phi', 'eta1', 'eta2', 'mll', 'pt1', 'pt2'] how features are returned when loaded

reference = BuildSample_DY(N_Events=3735456, INPUT_PATH='/data/marcol/HEPDATA/DILEPTON/DiLepton_SM/', rng=np.random.default_rng(seed=1234))
#reference = BuildSample_DY_2('/data/marcol/HEPDATA/DILEPTON/DiLepton_SM/', 1234, features=['delta_phi', 'eta1', 'eta2', 'mll', 'pt1', 'pt2'], N_Events=-1, nfiles=66, shuffle=True)

DiLepton_SM
(3735456, 6)


In [ ]:
mask_idx = np.where((reference[:, 4] <= cut_pt) | (reference[:, 5] <= cut_pt) | (np.abs(reference[:, 1]) > cut_eta) | (np.abs(reference[:, 2]) > cut_eta) | (reference[:, 3] <= cut_mll))[0]
masked_ref = np.delete(reference, mask_idx, axis=0)
#mask = 1*(reference[:,3]>=cut_mll)*(np.abs(reference[:,1])<cut_eta)*(np.abs(reference[:,2])<cut_eta)*(reference[:,4]>=cut_pt)*(reference[:,5]>=cut_pt)
#masked_ref = reference[mask>0]

In [ ]:
#masked_ref = masked_ref[:,]
mean_R = np.mean(masked_ref, axis=0)
std_R  = np.std(masked_ref, axis=0)

In [ ]:
mean_R

array([3.05212537e+00, 4.59876795e-04, 3.66978211e-04, 9.11198636e+01,
       4.67488118e+01, 3.60544255e+01])

In [ ]:
std_R


array([ 0.52342366,  1.24487457,  1.25051629, 15.86122412,  8.95046637])

In [ ]:
np.mean(masked_ref[:,1])

0.0004598767946975146

In [ ]:
np.mean(masked_ref[:,3])

91.11986355771019

In [ ]:
with open("./output/Ntoys100_NR200000_NB2000_NS90_nonres/t.txt", "r") as fp:
    # Load the dictionary from the file
    t_dict = json.load(fp)

In [ ]:
t_array = []
for key in t_dict.keys():
    t_array.append(t_dict[key])
t_array = np.array(t_array).transpose()

In [ ]:
np.save("./output/Ntoys100_NR200000_NB2000_NS90_nonres/t_array.npy",t_array)